In [38]:
import numpy as np
import pandas as pd

In [40]:
df = pd.read_csv('gurgaon_properties_post_feature_selection.csv')

In [42]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0.0,65.0,3,3,2.0,1.0,1766.0,0,0,0,1.0,1.0,2.60
1,0.0,65.0,3,3,3.0,3.0,1592.0,0,0,2,1.0,0.0,2.10
2,0.0,89.0,3,3,3.0,4.0,2129.0,1,0,0,1.0,0.0,2.60
3,0.0,74.0,2,2,3.0,3.0,1222.0,0,0,0,1.0,1.0,1.23
4,0.0,38.0,3,2,2.0,1.0,1444.0,0,0,0,2.0,2.0,1.35


In [44]:
X = df.drop(columns=['price'])
y = df['price']

In [46]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [48]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [50]:
columns_to_scale = ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']

In [52]:
# Applying the log1p transformation to the target variable, as it is rightly skewd
y_transformed = np.log1p(y)

In [54]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), columns_to_scale),
        ('cat', OneHotEncoder(drop='first'), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [56]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [58]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [60]:
scores.mean()

0.8858338674515392

In [62]:
scores.std()

0.025193480889300943

In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [66]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [68]:
y_pred = pipeline.predict(X_test)

In [70]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),np.expm1(y_pred))

0.5133739034782483